# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Batch Processing** </center>
---

**Date**: October, 2025

**Student Name**: Jaime Enrique Galindo Villegas

**Professor**: Pablo Camarillo Ramirez

# Introducción: Análisis de Transacciones en Mercado Libre
La aplicación web y móvil de Mercado libre procesa millones de transacciones diarias que pueden ser pedidos de artículos, suscripciones, pagos, etc. En el 2023 la plataforma logró ventas de 14,500 millones de dólares, con picos de demanda en momentos significativos del año como en el "Buen Fin" o el "Hot Sale" en los que sus ventas se disparan, en el 2024 llegaron a picos de más de 70% más ventas en el buen fin respecto a años anteriores.  

Este volumen masivo de datos, generado en multiples formatos presenta un desafio. Para abordarlo implementaré un pipeline de datos que procesa la información de órdenes de compra, con el objetivo de transformar los datos crudos en un formato limpio y estructurado que facilite posteriores analisis.

# Dataset: Órdenes de Compra

## Modelo de Datos
Para este problema, el **modelo de datos** más adecuado es el de **documento**.

Esto debido a que, según el diseño de pipeline que cree anteriormente, una orden de compra de ML es una entidad (JSON) que contiene toda su información de manera autocontenida, el usuario, monto, detalles de envio y articulos comprados. Por esto el modelo de documento es ideal al permitir presentar esta estructura jerárquica.

Esto es diferente de un modelo **relacional** ya que tendria que normalizar y dividir la información en multiples tablas como `Ordenes`, `Usuarios`, `Productos`, etc. Requeriria operaciones más complejas y, debido a la necesidad de procesar millones de transacciones, es mejor un modelo de **documento** que es más naturalmente compatible


## Generación del Dataset y Esquema
Para simular el flujo de datos de ML, se generará un archivo json (`orders.json`) usando la libreria Faker, en el que cada linea representará un objeto JSON de una orden de compra. Así simularé un sistema de ingesta en el que se recibirian los datos en tiempo real.

El **esquema** de cada objeto JSON generado será el siguiente:

- **order_id**: `String` - Identificador único de la orden.
- **timestamp**: `String` - Fecha y hora de la transacción en formato ISO.
- **user**: `Struct` - Objeto usuario.
  - **user_id**: `String` - Identificador del usuario.
  - **region**: `String` - Región de la compra (ej. "GDL").
  - **payment_method**: `String` - Método de pago utilizado (ej. "MP").
- **items**: `Array` - Lista de artículos comprados.
  - **item_id**: `String` - ID del producto.
  - **title**: `String` - Nombre del producto.
  - **category**: `String` - Categoría del producto.
  - **quantity**: `Integer` - Cantidad de unidades.
  - **unit_price**: `Double` - Precio original por unidad.
  - **final_price**: `Double` - Precio final por unidad con descuento.
  - **discount_applied**: `Boolean` - Indica si se aplicó descuento.
- **total_amount**: `Double` - Monto total de la orden.
- **shipping**: `Struct` - Información de envío.
  - **logistics_provider**: `String` - Empresa de paquetería.
  - **warehouse_origin**: `String` - Almacén de origen.
  - **estimated_delivery**: `String` - Fecha estimada de entrega.
  - **tracking_id**: `String` - Número de seguimiento.

# Transformations and Actions

# Persistence Data

# DAG